<a href="https://colab.research.google.com/github/rilschultz/CS167Notes/blob/main/Notebook4_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook \#4

**Part 1: [3 points]** You must run at least 6 variations of the algorithms and display their results using an __appropriate regression metric__ (again, use the scikit-learn modules). I will be looking for the following to be included in your comparison:

* **weighted k-Nearest-Neighbor** with a **small value of k** (the same one you used for the unweighted version)
* **weighted k-Nearest-Neighbor** with a **large value of k **(the same one you used for the unweighted version)
* a **decision tree** with default parameter values
* a **decision tree**, setting some kind of parameter that results in a smaller tree 
* a **Random Forest**, with default parameter values
* a **Random Forest**, with a **change to the number of trees** used.

You will need to use the documentation for sklearn for this Notebook. Here are some helpful links:
* [K Neighbors Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
* [Decision Tree Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) 


In [13]:
# load in the data and necessary libraries
import sklearn
import pandas
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler

from google.colab import drive
drive.mount('/content/drive')
songs = pandas.read_csv('/content/drive/MyDrive/datasets/spotify.csv') # change this to match your dataset directory
songs.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,...,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,...,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN


In [14]:
# Split the data into the training data and testing data
# we're only going to use a subset of the columns that are numeric
target= 'danceability'
predictors = ['energy', 'key', 'loudness','speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms']
train_data, test_data, train_sln, test_sln = train_test_split(songs[predictors], songs[target], test_size = 0.2, random_state=41)
train_data.head()

,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
11181,0.856,1,-3.748,0.4100,0.1500,0.00000,0.0573,0.754,160.042,126290
24502,0.956,5,-6.746,0.0563,0.0113,0.87600,0.1120,0.188,128.006,264666
18293,0.667,4,-5.191,0.0508,0.5860,0.00000,0.1160,0.385,128.962,241688
17867,0.443,6,-10.216,0.0325,0.9040,0.00197,0.0718,0.338,187.950,208253
16641,0.585,10,-5.466,0.0332,0.5530,0.00000,0.1950,0.238,215.168,214884


In [15]:
# w-kNN with small k
from sklearn import neighbors
# create our model
knn = neighbors.KNeighborsRegressor(n_neighbors=7, weights = 'distance')

# fit (train) the model to the data
knn.fit(train_data, train_sln)

# use the trained model to get predictions from our test_data
preds = knn.predict(test_data)
# use a metric to see how good our predictions are
print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.1990333170913955
MAE:  0.09522283758746254
MSE:  0.019655536298997765


In [18]:
# w-kNN with large k
from sklearn import neighbors
# create our model
knn = neighbors.KNeighborsRegressor(n_neighbors=80, weights = 'distance')

# fit (train) the model to the data
knn.fit(train_data, train_sln)

# use the trained model to get predictions from our test_data
preds = knn.predict(test_data)
# use a metric to see how good our predictions are
print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.31114722647333803
MAE:  0.08937404338467245
MSE:  0.016904287011727758


In [19]:
# decision tree with default paramters
from sklearn import neighbors
# create our model
dt = tree.DecisionTreeRegressor()

# fit (train) the model to the data
dt.fit(train_data, train_sln)

# use the trained model to get predictions from our test_data
preds = dt.predict(test_data)
# use a metric to see how good our predictions are
print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.31009185838734177
MAE:  0.08358582909821535
MSE:  0.016930185499349958


In [26]:
# decision tree with some kind of parameter that (hopefully) results in a smaller tree

from sklearn import neighbors
# create our model
dt = tree.DecisionTreeRegressor(max_depth = 2)

# fit (train) the model to the data
dt.fit(train_data, train_sln)

# use the trained model to get predictions from our test_data
preds = dt.predict(test_data)
# use a metric to see how good our predictions are
print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.24800958255847494
MAE:  0.10752735691074701
MSE:  0.018453670123763387


In [27]:
# Random Forest with default parameters
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
forest.fit(train_data, train_sln)
preds = forest.predict(test_data)

print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.6488371175778512
MAE:  0.06556193038647914
MSE:  0.008617455544148802


In [30]:
# Random Forest with change to the number of trees
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=30)
forest.fit(train_data, train_sln)
preds = forest.predict(test_data)

print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.6364925646238613
MAE:  0.06659929007603516
MSE:  0.008920388005460347


## Part 2: w-kNN on Normalized data
**2.)** Normalize the data and run a weighted k-Nearest Neighbors algorithm on it (from sklearn,  not the one we wrote from scratch). You can choose the k value. To Normalize, use the StandardScalar from sklearn.

In [31]:
# normalize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_data)
train_data_normalized = scaler.transform(train_data)
test_data_normalized = scaler.transform(test_data)
train_data_normalized
# Try using a decision tree on your normalized data
dt = tree.DecisionTreeRegressor()
dt.fit(train_data_normalized, train_sln)
predictions = dt.predict(test_data_normalized)
print("R2: ", metrics.r2_score(test_sln, predictions))
print("MAE: ", metrics.mean_absolute_error(test_sln, predictions))
print("MSE: ", metrics.mean_squared_error(test_sln, predictions))

# run a w-knn
knn = neighbors.KNeighborsRegressor(weights = 'distance')

# fit (train) the model to the data
knn.fit(train_data_normalized, train_sln)

# use the trained model to get predictions from our test_data
preds = knn.predict(test_data_normalized)
# use a metric to see how good our predictions are
print("R2: ", metrics.r2_score(test_sln, preds))
print("MAE: ", metrics.mean_absolute_error(test_sln, preds))
print("MSE: ", metrics.mean_squared_error(test_sln, preds))

R2:  0.3200448791664514
MAE:  0.08276914076350314
MSE:  0.01668594068195249
R2:  0.5599060257217434
MAE:  0.06825985991088436
MSE:  0.010799803875716904


## Part 3:
**3.)**Use a Markup cell to answer the following questions:
* What algorithm performed better? w-kNN, Decision Trees, or Random Forests? Why do you think this was the case?
* What effect did normalizing the data have on your results? Explain. 

The best algorithms were the random forests that produced around a 60% accuracy and weighting the knn on distance does increase it by a lot as well as normalizing it. When normalized the knn will have a similar result to the forest but the tree will continue to have the same results. For the single tree it should not have much of an effect because in the end the similar decisions will be made but it will have a bigger effect on the knn that would plot points differently.